In [51]:
import numpy as np
import igl
from scipy import sparse 
import h5py
import meshio as mio
import scipy

In [52]:
bilap_file = "../../build/icosphere_bilaplacian_nodes.obj"
v, _, _, f, _, _ = igl.read_obj(bilap_file)

In [66]:
L = igl.cotmatrix(v, f)
M = igl.massmatrix(v, f)

# assemble M_inv
M_inv_rows = np.array([i for i in range(M.shape[0])])
M_inv_cols = np.array([i for i in range(M.shape[1])])
M_inv_data = np.array([1.0/M[i,i] for i in M_inv_rows])
size = len(M_inv_cols)
M_inv = sparse.csc_matrix((M_inv_data, (M_inv_rows, M_inv_cols)), shape=(size, size))

A = (-L) @ M_inv @ (-L)

TypeError: cotmatrix(): incompatible function arguments. The following argument types are supported:
    1. (v: numpy.ndarray, f: numpy.ndarray) -> object

Invoked with: array([[-1.12148427, -1.17042627, -1.17042627],
       [-1.12148427, -1.17042627, -0.83601876],
       [-1.12148427, -1.17042627, -0.50161126],
       ...,
       [-0.5857096 ,  0.59181543,  0.45837439],
       [-0.45920594,  0.67214076,  0.46946989],
       [-0.43508849,  0.73501765,  0.41761272]]), <Closed HDF5 file>

In [ ]:
L = igl.cotmatrix(v, f)
# M = igl.massmatrix(v, f)
M = np.identity(L.shape[1])

# # assemble M_inv
# M_inv_rows = np.array([i for i in range(M.shape[0])])
# M_inv_cols = np.array([i for i in range(M.shape[1])])
# M_inv_data = np.array([1.0/M[i,i] for i in M_inv_rows])
# size = len(M_inv_cols)
# M_inv = sparse.csc_matrix((M_inv_data, (M_inv_rows, M_inv_cols)), shape=(size, size))

A = (-L) @ M_inv @ (-L)

In [63]:


local2global = np.loadtxt("../../build/icosphere_surface_correct_tri_to_tet_v_map.txt").astype(np.int32)
m = mio.read("../../build/test_linear_high_order_tetmesh.msh")
v = m.points
b = -(A @ v[local2global, :])

A_rows, A_cols, A_values = scipy.sparse.find(A)


with h5py.File("softj.hdf5", 'w') as f:
    f.create_dataset("b", data=b)
    f.create_dataset("A_triplets/values", data=A_values)
    f.create_dataset("A_triplets/cols", data=A_cols)
    f.create_dataset("A_triplets/rows", data=A_rows)

    f.create_dataset("weight", data=1e9)

In [59]:
A_

array([ 2.24138661e+04,  1.83449172e+03,  1.02408583e+01, ...,
        2.44864355e+02, -1.95889651e+03,  3.67292774e+03])

In [135]:
bilap_file = "../../build/icosphere_bilaplacian_nodes.obj"
v, _, _, f, _, _ = igl.read_obj(bilap_file)

In [136]:
def lap(v, f):
    a = igl.adjacency_matrix(f)
    # print(a)
    # Sum each row
    a_sum = np.sum(a, axis=1)
    a_sum = np.array(a_sum)[:,0]
    # print(a_sum.shape)
    # print(a_sum.shape)
    # a_sum_p = np.array([l[0] for l in a_sum])
    # Convert row sums into diagonal of sparse matrix
    a_diag = sparse.diags(a_sum)
    # print(a_diag)
    # a_diag = sparse.diags(a_sum_p)
    # Build uniform laplacian
    u = a - a_diag

    u = u / a_sum
    # print(u)

    return u

u = lap(v, f)
# print(u)

In [141]:
local2global = np.loadtxt("../../build/icosphere_surface_correct_tri_to_tet_v_map.txt").astype(np.int32)
m = mio.read("../../build/test_linear_high_order_tetmesh.msh")
v = m.points
A = u.copy()
b = -(A @ v[local2global, :])

A_p = A.tocoo(True)
print(A_p.data)

# A_rows, A_cols, A_values = scipy.sparse.find(A)
# print(A_values)
# print(A)

with h5py.File("softj.hdf5", 'w') as file:
    file.create_dataset("b", data=b)
    file.create_dataset("A_triplets/values", data=A_p.data)
    file.create_dataset("A_triplets/cols", data=A_p.col)
    file.create_dataset("A_triplets/rows", data=A_p.row)
    
    file.create_dataset("weight", data=1e9)


[-1.          0.1         0.1        ...  0.33333333  0.33333333
 -1.        ]


In [190]:
def bilap(v, f):
    L = igl.cotmatrix(v, f)
    M = igl.massmatrix(v, f)

    # assemble M_inv
    M_inv_rows = np.array([i for i in range(M.shape[0])])
    M_inv_cols = np.array([i for i in range(M.shape[1])])
    M_inv_data = np.array([1.0/M[i,i] for i in M_inv_rows])
    size = len(M_inv_cols)
    M_inv = sparse.csc_matrix((M_inv_data, (M_inv_rows, M_inv_cols)), shape=(size, size))

    return M_inv @ L


bilap_file = "../../build/icosphere_bilaplacian_nodes.obj"
vvv, _, _, f, _, _ = igl.read_obj(bilap_file)
xx = bilap(vvv, f)

local2global = np.loadtxt("../../build/icosphere_surface_correct_tri_to_tet_v_map.txt").astype(np.int32)
m = mio.read("../../build/test_linear_high_order_tetmesh.msh")
v = m.points

lamb = 1e-3

A = sparse.identity(xx.shape[0]) - lamb * xx.copy()
b = lamb * xx.copy() * v[local2global, :]
# b = -(A @ v[local2global, :])
d_r = scipy.sparse.linalg.spsolve(A, b)

V_r =  v[local2global, :] + d_r
igl.write_obj("test_lap.obj", V_r, f)

A_p = A.tocoo(True)
# print(A_p.data)

A_rows, A_cols, A_values = scipy.sparse.find(A)
# print(A_values)
# print(A)

with h5py.File("soft_i_1e_3.hdf5", 'w') as file:
    file.create_dataset("b", data=b)
    file.create_dataset("A_triplets/values", data=A_p.data)
    file.create_dataset("A_triplets/cols", data=A_p.col)
    file.create_dataset("A_triplets/rows", data=A_p.row)
    file.create_dataset("local2global", data=local2global.astype(np.int32))
    
    # file.create_dataset("weight", data=1e9)

In [181]:
V = vvv.copy()

for i in range(1000):
    V = V + 1e-4 * xx @ V 

In [182]:
igl.write_obj("test_lap.obj", V, f)

True